In [1]:
import torch
from torch import nn
import numpy as np
from torchvision import datasets
from torchvision import transforms
import torch.nn.functional as F

In [89]:
# Hyper Params
input_size = 3*32*32 # размер изображения в точках * количество цветов
num_classes = 10     # 10 видов изображений
batch_size = 16
n_epochs = 2
lr = 0.01

In [90]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [50]:
cifar_trainset = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
cifar_testset = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)
print(len(cifar_trainset))
print(len(cifar_testset))

Files already downloaded and verified
Files already downloaded and verified
50000
10000


In [51]:
train_loader = torch.utils.data.DataLoader(dataset=cifar_trainset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=cifar_testset, batch_size=batch_size, shuffle=False)

In [52]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [53]:
def make_train_step(model, loss_fn, optimizer):
    def train_step(X, y):
        model.train()
        yhat = model(X)
        loss = loss_fn(yhat, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        return loss
    return train_step

In [91]:
class CifarModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=0)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=0)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=0)
        self.pool = nn.MaxPool2d(2, stride=2)
        self.fc1 = nn.Linear(128 * 2 * 2, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.drop_out = nn.Dropout()

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 2 * 2)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [92]:
model = CifarModel()
model.to(device)

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

train_step = make_train_step(model, loss_fn, optimizer)

for epoch in range(n_epochs):
    print(f"Epoch No {epoch}")
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        loss = train_step(images, labels)

print(f"Model Loss {loss}")

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Точность: {round(correct / total * 100, 2)}")

Epoch No 0
Epoch No 1
Model Loss 2.2974352836608887
Точность: 10.0


In [37]:
class_correct = list(0. for i in range(10))
class_total = class_correct.copy()
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(4):
    print(f"Точность для {labels[i]} : {100 * class_correct[i] / class_total[i]}%")

Точность для 7 : 69.16299559471365%
Точность для 5 : 69.63562753036437%
Точность для 8 : 39.20863309352518%
Точность для 0 : 41.224489795918366%


In [79]:
n_epochs2 = 10
model = CifarModel()
model.to(device)

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

train_step = make_train_step(model, loss_fn, optimizer)

for epoch in range(n_epochs2):
    print(f"Epoch No {epoch}")
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        loss = train_step(images, labels)

print(f"Model Loss {loss}")

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Точность: {round(correct / total * 100, 2)}")

Epoch No 0
Epoch No 1
Epoch No 2
Epoch No 3
Epoch No 4
Epoch No 5
Epoch No 6
Epoch No 7
Epoch No 8
Epoch No 9
Model Loss 0.335824191570282
Точность: 70.41


In [23]:
class_correct = list(0. for i in range(10))
class_total = class_correct.copy()
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(4):
    print(f"Точность для {labels[i]} : {100 * class_correct[i] / class_total[i]}%")

Точность для 3 : 58.1%
Точность для 5 : 72.4%
Точность для 1 : 52.3%
Точность для 7 : 58.9%
